In [7]:
from genericpath import exists
import pdb
from pickletools import uint8
from turtle import pd
import torchvision
import torch
import os
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
import torchvision
import torch
from models_pytorch_lightning.generalized_mask_rcnn_pl import LitMaskRCNN

# from torchknickknacks import modelutils
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from pytorch_grad_cam.utils.image import show_cam_on_image

import numpy as np
import cv2
import requests
import random
import glob
from skimage.measure import label, regionprops, regionprops_table
from skimage import data, filters, measure, morphology
import pandas as pd
import wandb
from tqdm import tqdm
from skimage import data
from skimage.color import rgb2hed, hed2rgb
from collections import Counter

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve

In [20]:
class GetExternalDataOutput():
    
    # TODO fix the visualization flags
    def __init__(self, model, test_input_path, detection_threshold):
        self.model = model
        self.test_input_path = test_input_path
        self.detection_threshold = detection_threshold
        self.class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']
        self.class_to_colors = {'Cored': (255, 0, 0), 'Diffuse' : (0, 0, 255), 'Coarse-Grained': (0,255,0), 'CAA':(225, 255, 0)}
        #TODO change this to nas location later
        self.result_save_dir= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/gladstone_results/model_results"
        self.colors = np.random.uniform(0, 255, size=(len(self.class_names), 3))
    

    def get_outputs_nms(self, input_tensor, model, score_threshold = 0.5, iou_threshold = 0.5):
        #mask_list = []
        #label_list = []
        #score_list =[]
        #box_list = []
        with torch.no_grad():
            # forward pass of the image through the modle
            outputs = model(input_tensor)
        boxes = outputs[0]['boxes']
        labels = outputs[0]['labels']
        scores = outputs[0]['scores']
        masks = outputs[0]['masks']
        #print(boxes.shape,labels.shape, masks.shape)
        # Apply score threshold
        keep = scores > score_threshold
        boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
        #print(boxes.shape,labels.shape, masks.shape)
        #print(keep)
        keep = torchvision.ops.nms(boxes, scores, iou_threshold)
        boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
        #print(keep)
        #print(boxes.shape,labels.shape, masks.shape)
        scores = list(scores.detach().cpu().numpy())
        #print(scores)
        masks = list((masks>0.5).detach().cpu().numpy())
        #masks = list((masks>0.5).squeeze().detach().cpu().numpy())
        #print(masks.shape)
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in boxes.detach().cpu()]
        labels = list(labels.detach().cpu().numpy())
        labels = [self.class_names[i-1] for i in labels]
        return masks, boxes, labels, scores



    def prepare_input(self, image):
        image_float_np = np.float32(image) / 255
        # define the torchvision image transforms
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(1024),
            torchvision.transforms.ToTensor(),
        ])
        input_tensor = transform(transforms.ToPILImage()(image))
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_tensor = input_tensor.to(device)
        # Add a batch dimension:
        input_tensor = input_tensor.unsqueeze(0)
        return input_tensor, image_float_np


    def draw_boxes(self, boxes, labels, classes, image):
        for i, box in enumerate(boxes):
            color = self.colors[labels[i]-1]
            cv2.rectangle(
                image,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                color, 2
            )
            cv2.putText(image, classes[i], (int(box[0]), int(box[1] + 30)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1,
                        lineType=cv2.LINE_AA)
        return image
    
    def change_to_numpy_array(self, input_tensor):
        input_tensor_cpu = input_tensor.squeeze(0).cpu()  # Remove batch dimension and move to CPU if necessary
        # Convert to NumPy array
        numpy_array = input_tensor_cpu.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization
        # Scale back to 0-255 range if needed
        numpy_array = (numpy_array * 255).astype(np.uint8)
        return numpy_array
    
    
    
    def draw_segmentation_map(self, image, masks, boxes, labels,scores):
        alpha = 1 
        beta = 0.6 # transparency for the segmentation map
        gamma = 0 # scalar added to each 
        segmentation_map = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        result_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        masks = [ masks[i].squeeze() for i in range(len(masks))]
        for i in range(len(masks)):

            # TODO fix the color segmentation masks
            red_map = np.zeros_like(masks[i]).astype(np.uint8)
            green_map = np.zeros_like(masks[i]).astype(np.uint8)
            blue_map = np.zeros_like(masks[i]).astype(np.uint8)
            
            # apply a randon color mask to each object
            rect_color = (0,0,0)
            color = self.colors[random.randrange(0, len(self.colors))]
            red_map[masks[i] == 1], green_map[masks[i] == 1], blue_map[masks[i] == 1]  = self.class_to_colors[labels[i]]
            result_masks[masks[i] == 1] = 255
            # combine all the masks into a single image
            # change the format of mask to W,H, C

            # segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
            #convert the original PIL image into NumPy format
            image = np.array(image)
            # convert from RGB to OpenCV BGR format
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # apply mask on the image
            # cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, image)
            # draw the bounding boxes around the objects
            cv2.rectangle(image, boxes[i][0], boxes[i][1], color=rect_color, 
                        thickness=2)
            # Get the centre coords of the rectangle-plaque-detection/src/visualizat
            x1 = boxes[i][0][0]
            y1 = boxes[i][0][1]
            x2 = boxes[i][1][0]
            y2 = boxes[i][1][1]
            x = int((x1 + x2) / 2)
            y = int((y1+y2) / 2)

            
            # put the label text above the objects
            cv2.putText(image , labels[i]+str(np.round(scores[i],2)), (x1, y1-20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                        thickness=2, lineType=cv2.LINE_AA)
            
            # Convert Back
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image, result_masks
    
    


    def generate_results(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.eval().to(device)
        output_dict = {}
        output_score = {}
        for img in self.test_input_path:
            if os.path.exists(img):
                image = np.array(Image.open(img))
                if image.shape[2] == 4:
                    image = image[:,:, :3]
                input_tensor, image_float_np = self.prepare_input(image)
                masks, boxes, labels, scores = self.get_outputs_nms(input_tensor, self.model, 0.25, 0.25)
                print(boxes)
                #print(masks[0].shape)
                image_1024 = self.change_to_numpy_array(input_tensor)
                result_img, result_masks = self.draw_segmentation_map(image_1024, masks, boxes, labels, scores)
                #print(result_masks[0].shape)
                detection_img_name = img.split("/")[-1] + "_detection.png"
                #img.split("/")[-4] +"_" + img.split("/")[-3] +"_"+ img.split("/")[-2]+"_"
                detection_save_path = os.path.join(self.result_save_dir, detection_img_name)
                print(detection_save_path)
                bgr_img = cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(detection_save_path, bgr_img)
                output_dict[img] = labels
                output_score[img]=scores
        return output_dict, output_score
            

In [21]:
image_dataset =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles"
image_folders =  os.listdir(image_dataset)
gt_train = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/train.csv"
gt_train_df = pd.read_csv(gt_train)
gt_train_df["image_folder"]= gt_train_df["imagename"].apply(lambda l: l.split("/")[0])
gt_train_df["image_name"]= gt_train_df["imagename"].apply(lambda l: l.split("/")[-1].replace(".jpg",""))


In [22]:
gt_train_df = gt_train_df[gt_train_df["negative"]!=1]
gt_train_df["tile_folder"] = gt_train_df["image_name"].apply(lambda l : l.split("_")[4] if l.startswith("NA_") else l.split("_")[2])
gt_train_df["1536_img_name"] = gt_train_df["image_name"].apply(lambda l : l.split("_")[5]+".jpg" if l.startswith("NA_") else l.split("_")[3]+".jpg")
gt_train_df["1536_crop"] = gt_train_df["image_name"].apply(lambda l : l.split("_")[6] if l.startswith("NA_") else l.split("_")[4])
gt_train_df["data_present"] = gt_train_df["image_folder"].apply(lambda l: 1 if l in image_folders else 0)
gt_train_df1 = gt_train_df[gt_train_df["data_present"]==1]
gt_train_df1["1536_image_path"] = gt_train_df1.apply(lambda l: os.path.join(image_dataset,l["image_folder"],"0",l["tile_folder"], l["1536_img_name"]) , axis=1)

/tmp/ipykernel_123828/1800511060.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_train_df1["1536_image_path"] = gt_train_df1.apply(lambda l: os.path.join(image_dataset,l["image_folder"],"0",l["tile_folder"], l["1536_img_name"]) , axis=1)


In [23]:
model_name = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"
model = LitMaskRCNN.load_from_checkpoint(model_name)
#test_image_list =  gt_train_df1["1536_image_path"].unique()[:]

['/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/amyloid_train/48283-3_UTSW_temporal_6E10_0_5_14.jpg']

test_image_list = ['/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles/NA4185-02_AB/0/0/10.jpg',
       '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles/NA4185-02_AB/0/0/4.jpg',
       '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles/NA4185-02_AB/0/10/0.jpg',
       '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles/NA4185-02_AB/0/10/23.jpg',
       '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles/NA4185-02_AB/0/10/27.jpg']
test_image_list =['/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/amyloid_train/48283-3_UTSW_temporal_6E10_0_5_14.jpg']
test_image_list =['/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Norm_test_image4.jpg/test_image4.jpg']
test_image_list =["/gladstone/finkbeiner/steve/work/data/npsad_data/monika/test_img6.jpg"]
explain = GetExternalDataOutput(model, test_input_path=test_image_list, detection_threshold=0.25)
output_df, output_scores = explain.generate_results()

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/fabric/utilities/cloud_io.py:55: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mahirwar/miniconda

[[(637, 374), (896, 571)], [(516, 161), (765, 331)]]
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/gladstone_results/model_results/test_img6.jpg_detection.png
